### Classificateur de texte basique (intro to Tf_hub)

Avec la base de données IMBD (50 000 avis sur des films pos/neg)
* Bases : https://www.tensorflow.org/tutorials/keras/text_classification * 

In [ ]:
import os 
import re 
import shutil
import string

import tensorflow as tf


In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

In [ ]:
os.listdir(dataset_dir)

In [ ]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

Les __aclImdb/train/pos__ et __aclImdb/train/neg__ contiennent de nombreux fichiers texte, dont chacun est une critique de film unique.

In [ ]:
sample = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample) as f:
    print(f.read())

Chargement du jeu de données dans un format adapté à l'entrainement pour tf :  
main_directory/  
...class_a/   
......a_text_1.txt  
......a_text_2.txt  
...class_b/  
......b_text_1.txt  
......b_text_2.txt  


In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [ ]:
test_dir = os.path.join(dataset_dir, 'test')
print(os.listdir(test_dir))

train_dir = os.path.join(dataset_dir, 'train')
print(os.listdir(train_dir))

On divise la base de données en 3 parties :

In [ ]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

In [ ]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print("Review", text_batch.numpy()[i])
        print("Label", label_batch.numpy()[i])

In [ ]:
raw_train_ds.take(1)

In [ ]:
raw_train_ds.class_names[0], raw_train_ds.class_names[1]

In [ ]:
# Ensemble de validation :
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

# Remarquer le changement 'validation' en subset

In [ ]:
# Ensemble de test :
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

#### Préparation de l'ensemble de données :

Standardiser, tokeniser et vectoriser les données à l'aide de la couche __tf.keras.layers.TextVectorization__

- La __standardisation__ fait référence au prétraitement du texte, généralement pour supprimer la ponctuation ou les éléments HTML afin de simplifier l'ensemble de données.  
- La __tokenisation__ fait référence à la division de chaînes en jetons (par exemple, la division d'une phrase en mots individuels, en la divisant sur des espaces)
- La __vectorisation__ fait référence à la conversion de jetons en nombres (vecteurs) afin qu'ils puissent être introduits dans un réseau de neurones. 

__TextVectorization__ est une couche qui permet d'accomplir toutes ces tâches.

* Comme vous l'avez vu ci-dessus, les avis contiennent diverses balises HTML comme \<br /> . __Ces balises ne seront pas supprimées par le standardiseur par défaut dans la couche TextVectorization__ (qui convertit le texte en minuscules et supprime la ponctuation par défaut, mais ne supprime pas le HTML). Vous écrirez une fonction de standardisation personnalisée pour supprimer le code HTML.

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    
    return tf.strings.regex_replace(stripped_html,
    '[%s]' % re.escape(string.punctuation),
    '')

In [ ]:
string.punctuation

In [ ]:
data =['This is, punctuation, A not well positionned mAJ..',
'Other data',
'And here we go',
'Phrase ']

default_tokenizer = tf.keras.layers.TextVectorization()
default_tokenizer.adapt(data)

In [ ]:
tok_text = default_tokenizer(data)
tok_text

In [ ]:
# On créé un ensemble de données sans label 
train_text = raw_train_ds.map(lambda text, label: text)

max_features = 10000
sequence_length = 250

tokenizer_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_sequence_length=sequence_length
)

# On créé le vocabulaire 
tokenizer_layer.adapt(train_text)

In [ ]:
text = next(iter(train_text)) # On sort un batch de texte
first_text = text[0]          # On prend le premier text du batch  

len(text), first_text

In [ ]:
text =  tf.expand_dims(first_text, -1)
text, tokenizer_layer(text)

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    vectorized_text = vectorize_layer(text)
    return vectorized_text, label

__tf.expand_dims(intput, axis)__ : Returns a tensor with a length 1 axis inserted at index axis.

In [ ]:
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]

print("Review: ", first_review)
print("Label: ", raw_train_ds.class_names[first_label])
print("Vectorized review: ",vectorize_text(first_review, first_label))

On vectorise chacun de nos avis sur un vecteur de taille 250, si l'avis est assez bref il y aura beaucoup de 0 dans ce vecteur, et inversement.
C'est à partir de cette forme de codage que l'analyse de "sentiment" sera effectuée.  

La grandeur 'max_features' correspond au nombre maximal de "mots" considérés dans notre "vectorize_layer". (Un terme plus correcte serait *ensemble de caractères*)


Il est possible de faire l'opération inverse (entier -> jeton) avec la commande __.get_vocabulary()['int']__ 

---- __Note :__ puisqu'il s'agit d'une classification en 2 classes, serait-il possible d'utiliser des SVM ? 

In [ ]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

In [ ]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

Optimisons les performances d'entrainement de l'algo (voir le notebook dédié) :

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

#### Création du réseau de neurones :

In [ ]:
embedding_dim = 20

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(max_features + 1, embedding_dim),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Activation('sigmoid')]
)

model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer='adam',
    metrics=tf.metrics.BinaryAccuracy(threshold=0.0)
)

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

In [ ]:
loss, accuracy = model.evaluate(test_ds)

print(f"Erreur : {loss}")
print(f"Précision: {accuracy}")

In [ ]:
data = ['This was a incredibly deceptive movie',
'I was expecting it to be bad, I was good',
'Nothing to deal with sentiment and movie here',
'That was incredible']

vect_data = vectorize_layer(data)
model.predict(vect_data)

### Exporter le modèle 

In [ ]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  tf.keras.layers.Activation('sigmoid')
])

export_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

In [ ]:
# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)